In [1]:
## Impute LapTime with KNN (Sequential data) ##


from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns

# Load dataset
df = pd.read_csv('f1_2019_to_2022_all_drivers_all_data.csv', low_memory=False)

# Convert 'LapTime' and sector times to seconds
time_columns = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
for col in time_columns:
    df[col] = pd.to_timedelta(df[col]).dt.total_seconds()

# Keep the original 'Driver' and 'Circuit' for EDA, Preprocessing be for one hot encoding
df['Original_Driver'] = df['Driver']
df['Original_Circuit'] = df['Circuit']

# Encode Rainfall, FreshTyre, IsAccurate, and IsPersonalBest to integer
df['Rainfall'] = df['Rainfall'].astype(int)
df['FreshTyre'] = df['FreshTyre'].astype(int)
df['IsAccurate'] = df['IsAccurate'].astype(int)

# Drop columns not related to lap time prediction
columns_to_drop = ['Time', 'LapStartTime', 'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
                   'PitOutTime', 'PitInTime', 'LapStartDate', 'Deleted', 'DeletedReason', 'FastF1Generated','IsPersonalBest', 'Sector3Time','Sector2Time','Sector1Time']
df.drop(columns=columns_to_drop, inplace=True) # Sector 3 times are deleted since it could shadow impact of weather features

# One-hot encoding
df = pd.get_dummies(df, columns=['Driver', 'Circuit', 'Compound', 'Team'])

# Feature Engineering: 

# Categorize weather and return numerical labels for models Initial: 0 , 25 , 19 
def categorize_weather(row):
    if row['Rainfall'] > 0:
        return 'Rainy'
    elif row['AirTemp'] > 27:
        return 'Hot'
    elif row['AirTemp'] > 20:
        return 'Warm'
    else:
        return 'Cool'
df['Weather_Category'] = df.apply(categorize_weather, axis=1)

# Keep for further steps 
df['Original_Weather_Category'] = df['Weather_Category']
# Apply one-hot encoding to the 'Weather_Category' column
df = pd.get_dummies(df, columns=['Weather_Category'])

# Create Track temperature category based on the result of clustering 
df['TrackTemp_Cat'] = pd.cut(df['TrackTemp'], bins=[18, 27, 34, 41, 50, np.inf], labels=['VERY_LOW', 'Low', 'Medium', 'High', 'VERY_HIGH'])
# Initial values : [10, 20, 30, 40, 45, np.inf], labels=['VERY_LOW', 'Low', 'Medium', 'High', 'VERY_HIGH'])
df['Original_TrackTemp_Cat'] = df['TrackTemp_Cat']

# One hot encoding for TrackTemp_cat
df = pd.get_dummies(df, columns=['TrackTemp_Cat'])

# Tyre Age Interaction with TrackTemp 
df['TyreAge_TrackTemp'] = df['TyreLife'] * df['TrackTemp']

df.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82418 entries, 0 to 82417
Data columns (total 119 columns):
 #    Column                               Dtype   
---   ------                               -----   
 0    DriverNumber                         int64   
 1    LapTime                              float64 
 2    LapNumber                            float64 
 3    Stint                                float64 
 4    SpeedI1                              float64 
 5    SpeedI2                              float64 
 6    SpeedFL                              float64 
 7    SpeedST                              float64 
 8    TyreLife                             float64 
 9    FreshTyre                            int64   
 10   TrackStatus                          float64 
 11   Position                             float64 
 12   IsAccurate                           int64   
 13   AirTemp                              float64 
 14   Humidity                             float64 
 15   

In [7]:
# Standardize numeric features

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.impute import KNNImputer


numeric_features = ['SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'WindDirection', 'WindSpeed','TrackStatus','Position','LapTime']
scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])


df = df.drop(['Original_Driver', 'Original_Circuit','Original_Weather_Category','Original_TrackTemp_Cat'], axis= 1)

# Impute missing values
imputer = KNNImputer(n_neighbors=5)
df[numeric_features] = imputer.fit_transform(df[numeric_features])

In [8]:
# 1. Separate LapTime as dry or wet(rainy) condition ( since lapTime of rainy day would be recognized as outliers)
# 2. Remove Outliers for dry condition LapTime
# 3. Build Combined LapTime df (Outliers for dry days are deleted)

# Flag for rainy conditions
df['IsRainy'] = df['Rainfall'].apply(lambda x: 1 if x > 0 else 0)

df_dry = df[df['IsRainy'] == 0]
#df for rainy days 
df_wet = df[df['IsRainy'] == 1]

# Remove Outliers for dry days
Q1_dry = df_dry['LapTime'].quantile(0.25)
Q3_dry = df_dry['LapTime'].quantile(0.75)
IQR_dry = Q3_dry - Q1_dry
lower_bound_dry = Q1_dry - 1.5 * IQR_dry
upper_bound_dry = Q3_dry + 1.5 * IQR_dry
#Only dry days (Outliers Removed)
df_dry_filtered = df_dry[(df_dry['LapTime'] >= lower_bound_dry) & (df_dry['LapTime'] <= upper_bound_dry)]



# Build combined df for both dry-rainy days (Outliers are removed with rainy days lapTimes) 
df_combined = pd.concat([df_dry_filtered, df_wet], ignore_index=True)

df_combined.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80338 entries, 0 to 80337
Data columns (total 116 columns):
 #    Column                               Dtype  
---   ------                               -----  
 0    DriverNumber                         int64  
 1    LapTime                              float64
 2    LapNumber                            float64
 3    Stint                                float64
 4    SpeedI1                              float64
 5    SpeedI2                              float64
 6    SpeedFL                              float64
 7    SpeedST                              float64
 8    TyreLife                             float64
 9    FreshTyre                            int64  
 10   TrackStatus                          float64
 11   Position                             float64
 12   IsAccurate                           int64  
 13   AirTemp                              float64
 14   Humidity                             float64
 15   Pressure         

In [9]:
nan_counts = df_dry_filtered.isna().sum()
# Print columns with NaN count more than 0
print(nan_counts[nan_counts >0])

Series([], dtype: int64)


In [10]:
nan_counts = df_wet.isna().sum()
# Print columns with NaN count more than 0
print(nan_counts[nan_counts >0])

Series([], dtype: int64)


In [11]:
nan_counts = df_combined.isna().sum()
# Print columns with NaN count more than 0
print(nan_counts[nan_counts >0])

Series([], dtype: int64)


In [17]:
# 1. df_dry_filtered lapTime

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = df_dry_filtered.drop(['LapTime'], axis=1)
y = df_dry_filtered['LapTime']

# Split Train and test set for a model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Fitting a RandomForest model
model = RandomForestRegressor(n_estimators=100,max_depth=20,min_samples_split=5,random_state=42)
# Best Parameters: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 300}

model.fit(X_train, y_train)

# Print RMSE value of the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = mse ** 0.5
print(f"RMSE: {rmse}")



RMSE: 0.129437323390995


In [18]:
# Shows Important features

importance = model.feature_importances_

# Sort the feature importance
sorted_indices = np.argsort(importance)[::-1]

for index in sorted_indices:
    print(f"{X_train.columns[index]}: {importance[index]}")

SpeedFL: 0.30395802536100397
Pressure: 0.16034657535707683
Circuit_Sakhir Grand Prix: 0.09164186643000156
Circuit_Azerbaijan Grand Prix: 0.06870676649167544
IsAccurate: 0.065383062081098
SpeedI2: 0.04417674787712382
Circuit_Singapore Grand Prix: 0.04161951900180132
SpeedI1: 0.03491994615721838
Circuit_Bahrain Grand Prix: 0.03094387880230795
Circuit_Chinese Grand Prix: 0.026462146351814562
SpeedST: 0.015750693086320672
AirTemp: 0.011295496644740546
Year: 0.010516532128255958
TyreAge_TrackTemp: 0.010399821312445285
Circuit_United States Grand Prix: 0.009925629160596956
Humidity: 0.008006774877993004
LapNumber: 0.007042060523096687
Circuit_Japanese Grand Prix: 0.006875816788975855
Circuit_British Grand Prix: 0.006847213527438432
Circuit_Saudi Arabian Grand Prix: 0.006643299464319564
TrackTemp: 0.004978505049046586
TyreLife: 0.0038466256595553233
Position: 0.0034001000424525585
Circuit_Miami Grand Prix: 0.0031315932821755054
TrackStatus: 0.002715821102438061
WindDirection: 0.00249009510497

In [14]:
# 2. wet days lapTime

X = df_wet.drop(['LapTime'], axis=1)
y = df_wet['LapTime']

# Correctly filling NaN values
y.fillna(y.mean(), inplace=True)


# Split Train and test set for a model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Fitting a RandomForest model
model = RandomForestRegressor(n_estimators=100,max_depth=20,min_samples_split=5,random_state=42)
# Best Parameters: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 300}

model.fit(X_train, y_train)

# Print RMSE value of the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = mse ** 0.5
print(f"RMSE: {rmse}")

# Shows Important features

importance = model.feature_importances_

# Sort the feature importance
sorted_indices = np.argsort(importance)[::-1]

for index in sorted_indices:
    print(f"{X_train.columns[index]}: {importance[index]}")



/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_6938/4085755269.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(y.mean(), inplace=True)


RMSE: 0.24517512242183223
SpeedST: 0.3889173308827804
Humidity: 0.15804984655913926
SpeedFL: 0.12986055488079445
Pressure: 0.08421825633216533
Circuit_Belgian Grand Prix: 0.06972807311928127
IsAccurate: 0.06832298503376938
SpeedI2: 0.017218949900630304
SpeedI1: 0.013064965483196465
Compound_INTERMEDIATE: 0.012485947642983433
TrackTemp: 0.01066219661881858
TyreAge_TrackTemp: 0.008252637654131294
LapNumber: 0.007189661110637737
Position: 0.005378934803708045
AirTemp: 0.005017596446060868
WindSpeed: 0.00430965161840882
WindDirection: 0.002933159641476641
TyreLife: 0.00290961919637705
TrackStatus: 0.002109892478432286
Circuit_Hungarian Grand Prix: 0.001620629070271362
Circuit_German Grand Prix: 0.0012337460085362546
DriverNumber: 0.0007873598883056048
Stint: 0.000715905246347951
Compound_WET: 0.000648209042345778
Year: 0.000523879674492505
Team_Alfa Romeo: 0.000313877231238052
Circuit_Russian Grand Prix: 0.00030439163360332567
Driver_STR: 0.00016810695750536135
Circuit_Emilia Romagna Grand

In [15]:
# 3. Combined lapTime

X = df_combined.drop(['LapTime'], axis=1)
y = df_combined['LapTime']

y.fillna(y.mean(), inplace=True)

# Split Train and test set for a model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Fitting a RandomForest model
model = RandomForestRegressor(n_estimators=100,max_depth=20,min_samples_split=5,random_state=42)
# Best Parameters: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 300}

model.fit(X_train, y_train)

# Print RMSE value of the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = mse ** 0.5
print(f"RMSE: {rmse}")

# Shows Important features
importance = model.feature_importances_

# Sort the feature importance
sorted_indices = np.argsort(importance)[::-1]

for index in sorted_indices:
    print(f"{X_train.columns[index]}: {importance[index]}")

RMSE: 0.1526772535424068
SpeedFL: 0.3031746793484447
Pressure: 0.14080288188675885
Circuit_Sakhir Grand Prix: 0.08159624978750693
IsAccurate: 0.06468753790903219
Circuit_Azerbaijan Grand Prix: 0.060479431218094994
SpeedI2: 0.042204631581673015
Humidity: 0.04128079578075655
SpeedI1: 0.04113806555730962
Circuit_Singapore Grand Prix: 0.03687097956735473
Circuit_Bahrain Grand Prix: 0.028061195895255176
SpeedST: 0.024090755172768884
Circuit_Japanese Grand Prix: 0.017209837896197958
Circuit_Chinese Grand Prix: 0.013352000119192388
Circuit_Monaco Grand Prix: 0.00962784095326818
TyreAge_TrackTemp: 0.009468119683733558
AirTemp: 0.009201286939541075
Year: 0.0074394505867555
LapNumber: 0.005602549882219197
Circuit_British Grand Prix: 0.005329117842529023
Circuit_Saudi Arabian Grand Prix: 0.005318140847096825
TrackTemp: 0.004858603788710571
Circuit_Miami Grand Prix: 0.00457471796841485
Circuit_Belgian Grand Prix: 0.0044535616353320595
Circuit_United States Grand Prix: 0.004427545501961543
TyreLife